In [21]:
pip install pymysql wget

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9657 sha256=060830d6ca5a610494f0776bd0048a0831722fa47a2d25a8d428a4b5157c9498
  Stored in directory: /home/liulu/.cache/pip/wheels/bd/a8/c3/3cf2c14a1837a4e04bd98631724e81f33f462d86a1d895fae0
Successfully built wget
Note: you may need to restart the kernel to use updated packages.


In [67]:
import pymysql
import time
import datetime
import json
import functools
import wget, tarfile
import os
from urllib import error

"""
json解析datatime
"""
class DateEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, datetime.datetime):
            return obj.strftime("%Y-%m-%d %H:%M:%S")
        else:
            return json.JSONEncoder.default(self, obj)
"""
创建文件夹
"""     
def mkd(url):
    if os.path.exists(url) == False:
        os.makedirs(url)

"""
创建根文件夹
"""
cur_time = time.strftime("%Y%m%d%H%M%S", time.gmtime())   
path_root = os.path.join("/opt/filebrowser/data/dataset", cur_time)
mkd(path_root)

"""
文件批量下载
"""
def down_load(uri_list, out_path):
    for uri in uri_list:
        try:
            wget.download(uri["uri"], out=out_path)
        except error.HTTPError:
            print("ERR: ", uri["uri"])

## 参数start_time,end_time设置按时间段导出,时间格式为%Y-%m-%d


In [68]:
start_time = "2020-12-25"
end_time = ""
base_sql = "select concat('https://ty.cutcckms.com/zhjl', b.RELATIVE_PATH, b.file_name) uri, b.create_date from zhjl_itemtaskinspect a \
            left join base_file_upload_info b on a.fileid = b.file_id \
            where a.del_flag = 0 and b.del_flag = 0" 
file_sql = ""
if len(start_time) == 0 or len(end_time) == 0:
    file_sql = base_sql
else:
    file_sql = base_sql + " and b.create_date>= '" + start_time + "' and b.create_date<='" + end_time + "'"

connection = pymysql.connect(
        host="rm-bp1buvmr77jdglr16lo.mysql.rds.aliyuncs.com",
        user="",
        passwd="",
        database="",
        port=3306,
        charset="utf8",
        cursorclass=pymysql.cursors.DictCursor)
cursor = connection.cursor()

# 任务模板
sql = "select id, code, name from zhjl_tasktemplate where del_flag = 0 and property = 2"
cursor.execute(sql)
template = cursor.fetchall()

for template_item in template:
    # 任务
    task_sql = "select id, code, name from zhjl_task where del_flag = 0 and templateid = " + str(template_item["id"])
    cursor.execute(task_sql)
    task = cursor.fetchall()
    template_item["task"] = task
    for task_item in task:
        # 检查项
        inspect_sql = "select id, code, name from zhjl_taskinspect where del_flag = 0 and taskid = " + str(task_item["id"])
        cursor.execute(inspect_sql)
        inspect = cursor.fetchall()
        task_item["inspect"] = inspect
        for inspect_item in inspect:
            # 图片文件存放路径
            path_file = os.path.join(str(template_item["id"]) +"_"+ template_item["name"], 
                                 str(task_item["id"]) + "_" + task_item["name"],
                                 str(inspect_item["id"]) + "_" + inspect_item["name"])
            temp_file = os.path.join(path_root, path_file)
            mkd(temp_file)
            # 图片
            image_sql = file_sql + " and a.inspectid = " + str(inspect_item["id"])
            cursor.execute(image_sql)
            image = cursor.fetchall()
            inspect_item["image"] = image
            down_load(image, temp_file)
connection.close()

ERR:  https://ty.cutcckms.com/zhjl/upload/userfiles/2021/2021-11-08/211108202023114540/211108202023313816.jpg
ERR:  https://ty.cutcckms.com/zhjl/upload/userfiles/2021/2021-11-10/211110035851305046/211110035851184350.jpg
ERR:  https://ty.cutcckms.com/zhjl/upload/userfiles/2021/2021-11-10/211110040530277836/211110040530038643.jpg
ERR:  https://ty.cutcckms.com/zhjl/upload/userfiles/2021/2021-11-15/211115062658355355/211115062658535455.jpg
ERR:  https://ty.cutcckms.com/zhjl/upload/userfiles/2021/2021-11-22/211122062005186552/211122062005587640.jpg
ERR:  https://ty.cutcckms.com/zhjl/upload/userfiles/2021/2021-11-30/211130221912030252/211130221912327185.jpg
ERR:  https://ty.cutcckms.com/zhjl/upload/userfiles/2021/2021-11-30/211130235551204088/211130235551383586.jpg
ERR:  https://ty.cutcckms.com/zhjl/upload/userfiles/2021/2021-12-05/211205220557383434/211205220557106164.jpg
ERR:  https://ty.cutcckms.com/zhjl/upload/userfiles/2021/2021-12-06/211206124551007085/211206124551245306.jpg
ERR:  http

KeyboardInterrupt: 

## 导出json数据，层级为：模板-任务-检查项目-图片网络路径

In [26]:
# date_today = time.strftime("%Y-%m-%d", time.gmtime())
# file_name = "inspect_image" + date_today + ".txt"
# json.dumps = functools.partial(json.dumps, cls=DateEncoder)
# with open(file_name, 'w') as f:
#      f.write(json.dumps(template, ensure_ascii=False))